In [1]:
import cartopy.crs as ccrs
import hvplot.xarray
import xarray as xr
import requests
from glob import glob
import pandas as pd
from herbie import Herbie
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import thread_map
wave_wind_mapping = {
    # Combined Wave Parameters
    "Hsig": "HTSGW",       # Hsig: Significant wave height -> HTSGW: Significant Height of Combined Wind Waves and Swell
    "PkDir": "DIRPW",      # PkDir: Peak wave direction -> DIRPW: Direction of Combined Wind Waves and Swell (Often peak direction is reported as the primary direction)
    "Tm01": "PERPW",       # Tm01: Mean wave period (m01) -> PERPW: Mean Period of Combined Wind Waves and Swell (Tm01 is a common representation for mean period)
    # Wind Parameters
    #"Windv_x": "UGRD",     # Windv_x: Horizontal component of wind velocity -> UGRD: U-component of wind
    #"Windv_y": "VGRD",     # Windv_y: Vertical component of wind velocity (Likely intended as the other horizontal component) -> VGRD: V-component of wind
}

In [11]:
for year in range(2000, 2020):
    dates = xr.open_dataset(f"NZ_GEFS_{year}.nc").time.to_pandas()
    #print(year, len(dates))
    expected = pd.date_range(f"{year}-01-01 03:00", f"{year+1}-01-01", freq="3h")
    missing = expected[~expected.isin(dates)]
    extra = dates[~dates.isin(expected)]
    if len(missing) > 0:
        print("Missing dates:", len(missing), missing)

Missing dates: 56 DatetimeIndex(['2000-01-01 03:00:00', '2000-01-01 06:00:00',
               '2000-01-01 09:00:00', '2000-01-01 12:00:00',
               '2000-01-01 15:00:00', '2000-01-01 18:00:00',
               '2000-01-01 21:00:00', '2000-01-02 00:00:00',
               '2000-01-02 03:00:00', '2000-01-02 06:00:00',
               '2000-01-02 09:00:00', '2000-01-02 12:00:00',
               '2000-01-02 15:00:00', '2000-01-02 18:00:00',
               '2000-01-02 21:00:00', '2000-01-03 00:00:00',
               '2000-01-03 03:00:00', '2000-01-03 06:00:00',
               '2000-01-03 09:00:00', '2000-01-03 12:00:00',
               '2000-01-03 15:00:00', '2000-01-03 18:00:00',
               '2000-01-03 21:00:00', '2000-01-04 00:00:00',
               '2000-01-04 03:00:00', '2000-01-04 06:00:00',
               '2000-01-04 09:00:00', '2000-01-04 12:00:00',
               '2000-01-04 15:00:00', '2000-01-04 18:00:00',
               '2000-01-04 21:00:00', '2000-01-05 00:00:00',
      

In [16]:
# Temporal extent: 20000108-20191231
H = Herbie("2009-10-19 03:00:00", model="gefs_wave_reforecast", member=0, save_dir="./GEFS", verbose=True)

✅ Found ┊ model=gefs_wave_reforecast ┊ product=GEFSv12/wave_reforecast ┊ 2009-Oct-19 03:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


In [17]:
H.grib

'https://noaa-nws-gefswaves-reforecast-pds.s3.amazonaws.com/GEFSv12/reforecast/2009/20091019/gridded/gefs.wave.20091019.c00.global.0p25.grib2'

In [4]:
H.inventory("HTSGW").head(10)

,grib_message,start_byte,end_byte,range,reference_time,valid_time,variable,level,forecast_time,search_this
4,5,2717211,3102065.0,2717211-3102065,2009-10-19 03:00:00,2009-10-19 03:00:00,HTSGW,surface,24 hour fcst,:HTSGW:surface:24 hour fcst
26,27,15760237,16140784.0,15760237-16140784,2009-10-19 03:00:00,2009-10-19 03:00:00,HTSGW,surface,27 hour fcst,:HTSGW:surface:27 hour fcst
48,49,28805901,29186614.0,28805901-29186614,2009-10-19 03:00:00,2009-10-19 03:00:00,HTSGW,surface,30 hour fcst,:HTSGW:surface:30 hour fcst
70,71,41887921,42268641.0,41887921-42268641,2009-10-19 03:00:00,2009-10-19 03:00:00,HTSGW,surface,33 hour fcst,:HTSGW:surface:33 hour fcst
92,93,54992503,55374205.0,54992503-55374205,2009-10-19 03:00:00,2009-10-19 03:00:00,HTSGW,surface,36 hour fcst,:HTSGW:surface:36 hour fcst
114,115,68120048,68501964.0,68120048-68501964,2009-10-19 03:00:00,2009-10-19 03:00:00,HTSGW,surface,39 hour fcst,:HTSGW:surface:39 hour fcst
136,137,81261333,81643709.0,81261333-81643709,2009-10-19 03:00:00,2009-10-19 03:00:00,HTSGW,surface,42 hour fcst,:HTSGW:surface:42 hour fcst
158,159,94409333,94791956.0,94409333-94791956,2009-10-19 03:00:00,2009-10-19 03:00:00,HTSGW,surface,45 hour fcst,:HTSGW:surface:45 hour fcst
180,181,107555508,107938853.0,107555508-107938853,2009-10-19 03:00:00,2009-10-19 03:00:00,HTSGW,surface,48 hour fcst,:HTSGW:surface:48 hour fcst
202,203,120713180,121096520.0,120713180-121096520,2009-10-19 03:00:00,2009-10-19 03:00:00,HTSGW,surface,51 hour fcst,:HTSGW:surface:51 hour fcst


In [5]:
Herbie("2009-10-19 03:00:00", model="gefs_wave_reforecast", member=0, save_dir="./GEFS").xarray(rf":(?:HTSGW|DIRPW|PERPW):surface:(?:anl|(?:3|6|9|12|15|18|21) hour fcst)", remove_grib=True, backend_kwargs=dict(decode_timedelta=False), overwrite=True)

✅ Found ┊ model=gefs_wave_reforecast ┊ product=GEFSv12/wave_reforecast ┊ 2009-Oct-19 03:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


🦨 No subsets found with search=':(?:HTSGW|DIRPW|PERPW):surface:(?:anl|(?:3|6|9|12|15|18|21) hour fcst)' in self.model='gefs_wave_reforecast' self.date=Timestamp('2009-10-19 03:00:00') self.fxx=0


FileNotFoundError: [Errno 2] No such file or directory: 'GEFS/gefs_wave_reforecast/20091019/subset_54efb1fe__gefs.wave.20091019.c00.global.0p25.grib2'

In [29]:
H.xarray(rf":(?:HTSGW|DIRPW|PERPW):surface:(?:anl|(?:3|6|9|12|15|18|21) hour fcst)", backend_kwargs=dict(decode_timedelta=False))

<xarray.Dataset> Size: 100MB
Dimensions:              (step: 8, latitude: 721, longitude: 1440)
Coordinates:
    time                 datetime64[ns] 8B 2000-12-31T03:00:00
  * step                 (step) float64 64B 0.0 3.0 6.0 9.0 12.0 15.0 18.0 21.0
    surface              float64 8B 1.0
  * latitude             (latitude) float64 6kB 90.0 89.75 89.5 ... -89.75 -90.0
  * longitude            (longitude) float64 12kB 0.0 0.25 0.5 ... 359.5 359.8
    valid_time           (step) datetime64[ns] 64B 2000-12-31T03:00:00 ... 20...
    gribfile_projection  object 8B None
Data variables:
    swh                  (step, latitude, longitude) float32 33MB nan ... nan
    dirpw                (step, latitude, longitude) float32 33MB nan ... nan
    perpw                (step, latitude, longitude) float32 33MB nan ... nan
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    model:                   gefs_wave_reforecast
    product:                 GEFSv12/wave_reforecast
    description:             NOAA Wave Ensemble Reforecast
    remote_grib:             https://noaa-nws-gefswaves-reforecast-pds.s3.ama...
    local_grib:              GEFS/gefs_wave_reforecast/20001231/subset_25ef68...
    search:                  :(?:HTSGW|DIRPW|PERPW):surface:(?:anl|(?:3|6|9|1...

In [7]:
def download_GEFS(date):
    try:
        H = Herbie(date, model="gefs_wave_reforecast", member=0, save_dir="./GEFS")
        ds = H.xarray(rf":(?:HTSGW|DIRPW|PERPW):surface:(?:anl|(?:3|6|9|12|15|18|21) hour fcst)")
        ds = ds.sel(longitude=slice(165, 180), latitude=slice(-33,-48))
        ds = ds.drop_vars("time").rename({"step": "time"}).assign_coords(time=ds.valid_time.values).drop_vars(["gribfile_projection", "surface", "valid_time"])
        return ds
    except:
        print(f"Failed to download data for {date}")
        return None

dsets = thread_map(
    download_GEFS,
    pd.date_range("2000-01-08", "2000-01-31", freq="D"),
    max_workers=32,
    desc="Downloading GEFS data",
    unit="day",
)
dsets = xr.concat(dsets, dim="time")
dsets

✅ Found ┊ model=gefs_wave_reforecast ┊ product=GEFSv12/wave_reforecast ┊ 2000-Jan-09 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=gefs_wave_reforecast ┊ product=GEFSv12/wave_reforecast ┊ 2000-Jan-13 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=gefs_wave_reforecast ┊ product=GEFSv12/wave_reforecast ┊ 2000-Jan-12 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=gefs_wave_reforecast ┊ product=GEFSv12/wave_reforecast ┊ 2000-Jan-10 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=gefs_wave_reforecast ┊ product=GEFSv12/wave_reforecast ┊ 2000-Jan-16 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=gefs_wave_reforecast ┊ product=GEFSv12/wave_reforecast ┊ 2000-Jan-18 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=gefs_wave_reforecast ┊ product=GEFSv12/wave_reforecast ┊ 2000-Jan-20 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=gefs_wave_reforecast ┊ product=GEFSv12/wave_reforecast ┊ 2000-Jan-11 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


/mnt/data/physicsnemo/.venv/lib/python3.10/site-packages/herbie/core.py:1106: UserWarning: Will not remove GRIB file because it previously existed.
  warnings.warn("Will not remove GRIB file because it previously existed.")


Failed to download data for 2000-01-09 00:00:00


/mnt/data/physicsnemo/.venv/lib/python3.10/site-packages/herbie/core.py:1106: UserWarning: Will not remove GRIB file because it previously existed.
  warnings.warn("Will not remove GRIB file because it previously existed.")


Failed to download data for 2000-01-12 00:00:00


...

Failed to download data for 2000-01-08 00:00:00


/mnt/data/physicsnemo/.venv/lib/python3.10/site-packages/herbie/core.py:1106: UserWarning: Will not remove GRIB file because it previously existed.
  warnings.warn("Will not remove GRIB file because it previously existed.")


Failed to download data for 2000-01-17 00:00:00


/mnt/data/physicsnemo/.venv/lib/python3.10/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
/mnt/data/physicsnemo/.venv/lib/python3.10/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
/mnt/data/physicsnemo/.venv/lib/python3.10/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coo

TypeError: can only concatenate xarray Dataset and DataArray objects, got <class 'NoneType'>

In [5]:
dsets

NameError: name 'dsets' is not defined

In [ ]:
dsets.swh.hvplot(
    groupby="time",
    widget_type="scrubber",
    widget_location="bottom",
    clim=(0, 4.5),
    width=500,
    cmap="spectral_r",
    geo=True,
    coastline=True
)

NameError: name 'dsets' is not defined